<a href="https://colab.research.google.com/github/dumanzhusupov/MoviesFromServerToDatabase/blob/main/PySparkML_PySparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Инфраструктура для моделей машинного обучения. Практическая работа

# Цель практической работы

Потренироваться в использовании библиотек PySpark SQL и PySpark ML для предобработки данных и обучения моделей.

# Что входит в практическую работу

1. Инициализация спарк-сессии.
2. Загрузка данных.
3. Ознакомление с данными.
4. Преобразование типов столбцов.
5. Очистка данных.
6. Feature-инжиниринг.
7. Векторизация фичей.
8. Создание и обучение модели.
9. Выбор лучшей модели.
10. Обратная связь.


# Что оценивается

- Пройдены все этапы работы.
- Спарк-сессия успешно запущена.
- Данные прочитаны.
- Все колонки с числовыми значениями преобразованы в числовые типы данных (Int или Double).
- Отфильтрованы все строки с Null-значениями.
- Созданы новые фичи.
- Все категориальные колонки преобразованы в числовой вид, выполнены все этапы векторизации признаков.
- Выборка разделена на обучающую и тестовую.
- Создано три объекта: модель, сетка гиперпараметров и evaluator.
- Создан объект класса CrossValidator и обучен на обучающей выборке.
- Выбрана лучшая модель, посчитана метрика качества лучшей модели.


# Задача

Используя данные о клиентах телекоммуникационной компании, обучите модель, предсказывающую их отток.

Описание данных, с которыми вы будете работать:

* **CustomerID**: ID клиента.
* **Gender**: пол клиента.
* **SeniorCitizen**: пенсионер ли клиент (1 — да, 0 — нет).
* **Partner**: есть у клиента партнёр (жена, муж) или нет (Yes/No).
* **Dependents**: есть ли у клиента инждивенцы, например дети (Yes/No).
* **Tenure**: как много месяцев клиент оставался в компании.
* **PhoneService**: подключена ли у клиента телефонная служба (Yes/No).
* **MultipleLines**: подключено ли несколько телефонных линий (Yes, No, No phone service).
* **InternetService**: интернет-провайдер клиента (DSL, Fiber optic, No).
* **OnlineSecurity**: подключена ли у клиента услуга онлайн-безопасности (Yes, No, No internet service)
* **OnlineBackup**: подключена ли услуга резервного копирования онлайн (Yes, No, No internet service).
* **DeviceProtection**: подключена ли услуга защиты устройства (Yes, No, No internet service)
* **TechSupport**: есть ли у клиента техническая поддержка (Yes, No, No internet service).
* **StreamingTV**: подключена ли услуга потокового телевидения (Yes, No, No internet service).
* **StreamingMovies**: подключена ли услуга стримингового воспроизведения фильмов (Yes, No, No internet service).
* **Contract**: тип контракта клиента (Month-to-month, One year, Two year).
* **PaperlessBilling**: есть ли безбумажный счёт.
* **PaymentMethod**: способ оплаты услуг (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)).
* **MonthlyCharges**: сумма, которая списывается ежемесячно.
* **TotalCharges**: сумма, списанная за всё время.
* **Churn**: ушёл ли клиент (Yes/No). Это целевая переменная, которую нужно предсказать.


# 1. Инициализация спарк-сессии

Инициализируйте спарк-сессию.

Эта ячейка нужна для того, чтобы заргузить необходимые библиотеки и настроить окружение Google Colab для работы со Spark.

Просто запустите её перед выполением задания :)

In [2]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')


In [3]:
! pip install pyarrow

In [4]:
from pyspark.sql import SparkSession
#/content/sample_data/WA_Fn-UseC_-Telco-Customer-Churn.csv

spark = SparkSession.builder.getOrCreate()
spark

### Ваш код здесь ###

# 2. Загрузка данных
Загрузите данные, сохраните их в переменную типа sparkDataframe, используя метод read.csv (не забывайте про header и delimiter).

In [6]:
### Ваш код здесь ###
data = spark.read.csv("/WA_Fn-UseC_-Telco-Customer-Churn.csv", header=True, inferSchema=True)
data.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|  

In [7]:
toDoubleColumns = ['MonthlyCharges', 'TotalCharges']
toIntColumns = ['tenure','SeniorCitizen']

In [8]:
data.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [9]:
data.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [10]:
data.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|  

In [11]:
for col in data.columns:
    print(col, data.filter(data[col].isNull()).count())

customerID 0
gender 0
SeniorCitizen 0
Partner 0
Dependents 0
tenure 0
PhoneService 0
MultipleLines 0
InternetService 0
OnlineSecurity 0
OnlineBackup 0
DeviceProtection 0
TechSupport 0
StreamingTV 0
StreamingMovies 0
Contract 0
PaperlessBilling 0
PaymentMethod 0
MonthlyCharges 0
TotalCharges 0
Churn 0


In [12]:
data.pandas_api().isna().sum()

/usr/local/lib/python3.11/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

# 3. Ознакомление с данными
1. Выведите на экран первые несколько строк датафрейма.


In [13]:
### Ваш код здесь ###
data.show(5)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    


2. Выведите общее количество строк датафрейма.



In [14]:
### Ваш код здесь ###
data.pandas_api().shape

(7043, 21)

3. Выведите структуру (схему) датафрейма.

In [15]:
### Ваш код здесь ###
data.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



# 4. Преобразование типов столбцов
Преобразуйте тип столбцов у числовых признаков (Int — если признак целочисленный, Double — если признак не целочисленный). Сохраните преобразованный датафрейм в новую переменную.

## Совет

Если вам сложно выполнить это задание, изучите дополнительные материалы: [об операторе Cast](https://sparkbyexamples.com/pyspark/pyspark-cast-column-type/), [об операторе Select](https://sparkbyexamples.com/pyspark/select-columns-from-pyspark-dataframe/).



In [16]:
from pyspark.sql.functions import expr, col

#Cast to Int
for col in toIntColumns:
  data = data.withColumn(col, data[col].cast("Int"))
#cast to Double
for col in toDoubleColumns:
  data = data.withColumn(col, data[col].cast("Double"))
#cast to Int
### Ваш код здесь ###

# 5. Очистка данных
Проверьте, есть ли в какой-либо колонке Null-значения. Для этого можно использовать your_dataframe.filter(col("colname")).isNull()).

Выведите на экран несколько строк с Null-значениями в одной из колонок.

Сохраните очищенный от строк с Null-значениями датафрейм в новую переменную. Для фильтрации этих значений можно использовать метод .isNotNull().

Колонок в датафрейме много, проверять каждую неудобно и долго. Подумайте, как упроситить эту работу, если использовать, например, перебор с циклом for.

[Примеры использования операторов isNull() и isNotNull()](https://sparkbyexamples.com/pyspark/pyspark-isnull/).


In [17]:
### Ваш код здесь ###

data.filter("TotalCharges is NULL").show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+----------------+--------------------+--------------+------------+-----+
|4472-LVYGI|Female|            0|    Yes|       Yes|     0|          No|No phon

In [18]:
data = data.fillna(0)

In [19]:

data.filter("TotalCharges is NULL").show()

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-

In [20]:
data.pandas_api().isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

# 6. Feature-инжиниринг
Добавьте в датафрейм одну или несколько новых фичей. Удалите колонки, которые, как вам кажется, нужно убрать из фичей. Обоснуйте свои решения.

Нужно точно удалить кастомер айди колонку, так как он не влияет на отток клиентов.
А из новых признаков которые будут вляит на отток или отказ клиента от услуг, будет признак который покажет является ли продливал ли клиент наши услуги в прошлом. Так как если после контракта уже продливал еще раз, то он склонен дальше продлевать, а если нет менее склонен. Будем считать их как: tenure >=  Длительность контракта в месяцах.(Берем только долгосроные контракты). Так же те кто взял долгосрочный контракт то он вряд-ли будет отазыватся, так как после долгосрочного контракта они менее склонны к оттоку.  

In [21]:
from pyspark.sql.functions import when

print(data.select(data["Contract"]).distinct().show())
data_edit = data.withColumn("Contract_lasting",
                when(data["Contract"] == "Two year", 24)
                .when(data["Contract"] == "One year", 12)
                .otherwise(0))
data_edit.show()
### Ваш код здесь ###

+--------------+
|      Contract|
+--------------+
|Month-to-month|
|      One year|
|      Two year|
+--------------+

None
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+----------------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|Contract_lasting|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+----------

In [22]:
data_edit = data_edit.drop("CustomerID").drop("Contract")
data_edit.show()

+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----------------+--------------------+--------------+------------+-----+----------------+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|Contract_lasting|
+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----------------+--------------------+--------------+------------+-----+----------------+
|Female|            0|    Yes|        No|     1|          No|No phone service|          

In [23]:
data_edit = data_edit.withColumn("isLoyal",
                     when(data_edit["Contract_lasting"] < data_edit["tenure"], 1)
                     .otherwise(0))

data_edit.show()

+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----------------+--------------------+--------------+------------+-----+----------------+-------+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|Contract_lasting|isLoyal|
+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----------------+--------------------+--------------+------------+-----+----------------+-------+
|Female|            0|    Yes|        No|     1|          No|No 

#7. Векторизация фичей
Подготовьте данные к обучению:





1. Преобразуйте текстовые колонки в числа, используя StringIndexer.
Удалите столбцы со старыми (непреобразованными) признаками. Выведите на экран структуру получившегося датафрейма. Не забывайте о столбце Churn. Хоть он и выступает в задаче как таргет, он имеет текстовый тип, поэтому тоже должен быть закодирован числовыми значениями.

Чтобы использовать StringIndexer для всех категориальных признаков сразу, а не для каждого отдельно, можно применить сущность pipeline.

**Пример кода:**

##### #Задаём список текстовых колонок:
text_columns = ["text_col_1", "text_col_2", "text_col_3"]

##### #Задаём список StringIndexer'ов — сущностей, каждая из которых будет кодировать одну текстовую колонку числами. Имена преобразованных колонок будут заканчиваться на _index:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index",).fit(<ваш датасет>) for column in text_columns]

##### #Создаём Pipeline из StringIndexer'ов:
pipeline = Pipeline(stages=indexers)

##### #Скармливаем нашему pipeline датафрейм, удаляя старые колонки:
new_dataframe = pipeline.fit(<ваш датасет>).transform(<ваш датасет>).drop(*text_columns)


In [24]:
from pyspark import mllib
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

#список колонок с текстовым типом
text_cols = ["gender", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", \
                "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", \
                "StreamingMovies", "PaperlessBilling", "PaymentMethod", "Churn"]

#
### Ваш код здесь ###
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index",).fit(data_edit) for column in text_cols]
pipeline = Pipeline(stages=indexers)
new_data = pipeline.fit(data_edit).transform(data_edit).drop(*text_cols)
new_data.show()

+-------------+------+--------------+------------+----------------+-------+------------+-------------+----------------+------------------+-------------------+---------------------+--------------------+------------------+----------------------+-----------------+-----------------+---------------------+----------------------+-------------------+-----------+
|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|Contract_lasting|isLoyal|gender_index|Partner_index|Dependents_index|PhoneService_index|MultipleLines_index|InternetService_index|OnlineSecurity_index|OnlineBackup_index|DeviceProtection_index|TechSupport_index|StreamingTV_index|StreamingMovies_index|PaperlessBilling_index|PaymentMethod_index|Churn_index|
+-------------+------+--------------+------------+----------------+-------+------------+-------------+----------------+------------------+-------------------+---------------------+--------------------+------------------+----------------------+-----------------+-----------------+-------

2. Векторизуйте категориальные признаки, используя OneHotEncoder.
Удалите столбцы со старыми (непреобразованными) признаками.
Выведите на экран структуру получившегося после преобразований датафрейма.


In [25]:
from pyspark.ml.feature import OneHotEncoder

#список категориальных колонок
features_inp  = ["MultipleLines", "InternetService", \
                "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", \
                "StreamingMovies", "PaymentMethod"]
features_inp = [col+"_index" for col in features_inp]

ohe = OneHotEncoder(inputCols=features_inp, outputCols=[x + "_ohe" for x in features_inp])
data_ohe = ohe.fit(new_data).transform(new_data).drop(*features_inp)
data_ohe.show()

### Ваш код здесь ###



+-------------+------+--------------+------------+----------------+-------+------------+-------------+----------------+------------------+----------------------+-----------+-----------------------+-------------------------+------------------------+----------------------+--------------------------+---------------------+---------------------+-------------------------+-----------------------+
|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|Contract_lasting|isLoyal|gender_index|Partner_index|Dependents_index|PhoneService_index|PaperlessBilling_index|Churn_index|MultipleLines_index_ohe|InternetService_index_ohe|OnlineSecurity_index_ohe|OnlineBackup_index_ohe|DeviceProtection_index_ohe|TechSupport_index_ohe|StreamingTV_index_ohe|StreamingMovies_index_ohe|PaymentMethod_index_ohe|
+-------------+------+--------------+------------+----------------+-------+------------+-------------+----------------+------------------+----------------------+-----------+-----------------------+-----------------

3. Объедините колонки фичей в один вектор, используя VectorAssembler.
Удалите столбцы со старыми (непреобразованными) признаками.
Выведите на экран первые несколько строк и структуру получившегося датафрейма.

In [26]:
from pyspark.ml.feature import VectorAssembler


### Ваш код здесь ###
assembler = VectorAssembler(inputCols=data_ohe.columns,outputCol="features")

out_data = assembler.transform(data_ohe)
out_data.show()

+-------------+------+--------------+------------+----------------+-------+------------+-------------+----------------+------------------+----------------------+-----------+-----------------------+-------------------------+------------------------+----------------------+--------------------------+---------------------+---------------------+-------------------------+-----------------------+--------------------+
|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|Contract_lasting|isLoyal|gender_index|Partner_index|Dependents_index|PhoneService_index|PaperlessBilling_index|Churn_index|MultipleLines_index_ohe|InternetService_index_ohe|OnlineSecurity_index_ohe|OnlineBackup_index_ohe|DeviceProtection_index_ohe|TechSupport_index_ohe|StreamingTV_index_ohe|StreamingMovies_index_ohe|PaymentMethod_index_ohe|            features|
+-------------+------+--------------+------------+----------------+-------+------------+-------------+----------------+------------------+----------------------+-----------

#8. Создание и обучение модели

1. Создайте модель — логистическую регрессию (используя LogisticRegression). В качестве параметров класса LogisticRegression укажите колонку фичей (параметр featuresCol), колонку-таргет (параметр labelCol) из датафрейма и имя колонки, в которую будут записываться предсказания (параметр predictionCol).

In [46]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="Churn_index", predictionCol="prediction")

### Ваш код здесь ###

2. Разделите датафрейм на обучающую и тестовую выборку.

In [47]:
train, test = out_data.randomSplit([0.8, 0.2], seed = 42)
### Ваш код здесь ###

3. Создайте объекты — сетки гиперпараметров для каждой модели, используя ParamGridBuilder. Так же, как и в ноутбуке из последнего видео, в сетку гиперпараметров можно добавить значения параметров regParam и elasticNetParam.

Вы можете ознакомиться [с документацией объекта LogisticRegression в PySpark](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.LogisticRegression.html) и добавить в сетку больше параметров.


In [48]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import numpy as np

### Ваш код здесь ###
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, np.arange(0.01, 2.0,0.1))
             .addGrid(lr.elasticNetParam, np.arange(0.0,1.0,0.1))
             .build())
#

4. Создайте объект evaluator, который будет отвечать за метрику качества при обучении. Для этого используйте класс BinaryClassificationEvaluator со следующими параметрами: rawPredictionCol — колонка с предсказаниями, labelCol — колонка с таргетом.

У вас, возможно, возник вопрос, какую метрику качества берёт по умолчанию BinaryClassificationEvaluator. По умолчанию BinaryClassificationEvaluator будет рассчитывать areaUnderROC. Это метрика оценки площади под кривой ROC (Receiver Operating Characteristic), которая служит графической интерпретацией производительности модели. Эта метрика качества находится в пределах от 0 до 1. Чем выше метрика, тем более качественные предсказания делает модель.

In [55]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Churn_index", metricName="areaUnderROC")



### Ваш код здесь ###

5. Создайте объект CrossValidator, в качестве параметров укажите уже созданные вами модель, сетку гиперпараметров и evaluator.

In [56]:
from pyspark.ml.tuning import CrossValidator

### Ваш код здесь ###
cross_validator = CrossValidator(estimator=lr,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=evaluator,
                                  numFolds=3)

In [57]:
train.show(5)

+-------------+------+--------------+------------+----------------+-------+------------+-------------+----------------+------------------+----------------------+-----------+-----------------------+-------------------------+------------------------+----------------------+--------------------------+---------------------+---------------------+-------------------------+-----------------------+--------------------+
|SeniorCitizen|tenure|MonthlyCharges|TotalCharges|Contract_lasting|isLoyal|gender_index|Partner_index|Dependents_index|PhoneService_index|PaperlessBilling_index|Churn_index|MultipleLines_index_ohe|InternetService_index_ohe|OnlineSecurity_index_ohe|OnlineBackup_index_ohe|DeviceProtection_index_ohe|TechSupport_index_ohe|StreamingTV_index_ohe|StreamingMovies_index_ohe|PaymentMethod_index_ohe|            features|
+-------------+------+--------------+------------+----------------+-------+------------+-------------+----------------+------------------+----------------------+-----------

6. Запустите обучение модели на тренировочной выборке. Сохраните обученную модель в новую переменную.

In [58]:
model = cross_validator.fit(train)
### Ваш код здесь ###

#9. Выбор лучшей модели

1. Выберите лучшую модель, сохраните её в отдельную переменную, отобразите её параметры.

Вывод параметров модели в PySpark можно сделать, используя метод extractParamMap().

In [59]:
best_model = model.bestModel
best_model.extractParamMap()
### Ваш код здесь ###

{Param(parent='LogisticRegression_785ae0f1ba60', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_785ae0f1ba60', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_785ae0f1ba60', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto',
 Param(parent='LogisticRegression_785ae0f1ba60', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='LogisticRegression_785ae0f1ba60', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_785ae0f1ba60', name='labelCol', doc='label column name.'): 'Churn_index',
 Param(parent='LogisticRegression_785ae0f1ba60', name='maxBlockSizeInMB', doc='maximum memory in MB

2. Запустите лучшую модель в режиме предсказания на тестовой выборке. Сохраните предсказания в отдельную переменную. Выведите первые несколько строк датафрейма с предсказаниями на экран.

Запуск модели в режиме предсказания выполняется при помощи метода .transform(<тестовая выборка>).

In [60]:
ypred = best_model.transform(test)
### Ваш код здесь ###

3. Получите метрику качества модели. Для этого примените к объекту evaluator метод .evaluate(<ваш датафрейм с предсказаниями>).



In [61]:
evaluator.evaluate(ypred)
### Ваш код здесь ###

1.0

In [73]:
lr2 = LogisticRegression(featuresCol="features", labelCol="Churn_index", predictionCol="prediction",elasticNetParam=0,regParam=0.01)

lr2 = lr2.fit(train)
evaluator.evaluate(lr2.transform(test))

1.0

#10. Обратная связь
Вы ознакомились с возможностями двух мощных библиотек: PySpark SQL для предобработки данных и PySpark ML для машинного обучения.

Поделитесь впечатлениями от работы с новыми библиотеками. В чём они более удобны, чем уже знакомые вам Pandas и Sklearn, а в чём нет.

Синтаксису нужно привыкнуть, ведь логик работы другая и отличаются реализаций и названия. А так если привыкнуть то классный инструмент. Обязательно надо попробовать реализовать проекты. Так же было бы интересно порешать задачи заточеные под инфраструктуру PySpark-а.

# Как отправить работу на проверку

Загрузите файл с заданиями, откройте его через Jupyter Notebook в Google Colab. Скачайте файл с датасетом и загрузите его в Colab. Выполните задачи, сохраните изменения: воспользуйтесь опцией Save and Checkpoint из вкладки меню File или кнопкой Save and Checkpoint на панели инструментов. Отправьте через форму ниже итоговый файл Jupyter Notebook (.ipynb) или ссылку на него.